In [930]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)

import xgboost
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import TweedieRegressor

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer

from prepare import x_y_split, rmse, select_kbest, rfe

In [1121]:
df = pd.read_csv('season.csv',index_col=0)

seasons = range(2016,2023)
positions = ['QB','RB','WR','TE']
add = []

df = df[df['g'] > 5]

for s in seasons:
    for p in positions:
        test = df[(df.year==s)&(df.pos==p)].sort_values('avg_draft_pos_ppr').reset_index(drop=True)
        test['adp_by_pos'] = test.index+1
        
        test = test[(test.year==s)&(test.pos==p)].sort_values('ppr_pts',ascending=False).reset_index(drop=True)
        test['pos_rank'] = test.index+1
        
        add.append(test)

df = pd.concat(add).reset_index(drop=True)

for i in range(0,len(df.index)):
    if df.loc[i,'pos'] =='QB' or df.loc[i,'pos'] =='TE':
        
        if df.loc[i, 'adp_by_pos'] <= 3:
            if df.loc[i,'pos_rank'] <= 3:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0
        else:
            if df.loc[i,'pos_rank']<=12:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

    else:
        if df.loc[i, 'adp_by_pos'] <= 6:
            if df.loc[i,'pos_rank'] <= 6:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

        elif df.loc[i, 'adp_by_pos'] > 36:
            if df.loc[i,'pos_rank'] < 36:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

        else:
            if df.loc[i,'pos_rank'] <= df.loc[i,'adp_by_pos']:
                df.loc[i, 'success'] = 1
            else:
                df.loc[i, 'success'] = 0
        
        
for i in range(0,len(df.index)):
    if df.loc[i,'avg_draft_pos_ppr'] <= 12:
        df.loc[i, 'round'] = 1
    elif df.loc[i,'avg_draft_pos_ppr'] <= 24:
        df.loc[i, 'round'] = 2
    elif df.loc[i,'avg_draft_pos_ppr'] <= 36:
        df.loc[i, 'round'] = 3
    elif df.loc[i,'avg_draft_pos_ppr'] <= 48:
        df.loc[i, 'round'] = 4
    elif df.loc[i,'avg_draft_pos_ppr'] <= 60:
        df.loc[i, 'round'] = 5
    elif df.loc[i,'avg_draft_pos_ppr'] <= 72:
        df.loc[i, 'round'] = 6
    elif df.loc[i,'avg_draft_pos_ppr'] <= 84:
        df.loc[i, 'round'] = 7
    elif df.loc[i,'avg_draft_pos_ppr'] <= 96:
        df.loc[i, 'round'] = 8
    elif df.loc[i,'avg_draft_pos_ppr'] <= 108:
        df.loc[i, 'round'] = 9
    elif df.loc[i,'avg_draft_pos_ppr'] <= 120:
        df.loc[i, 'round'] = 10
    elif df.loc[i,'avg_draft_pos_ppr'] <= 132:
        df.loc[i, 'round'] = 11
    elif df.loc[i,'avg_draft_pos_ppr'] <= 144:
        df.loc[i, 'round'] = 12
    elif df.loc[i,'avg_draft_pos_ppr'] <= 156:
        df.loc[i, 'round'] = 13
    elif df.loc[i,'avg_draft_pos_ppr'] <= 168:
        df.loc[i, 'round'] = 14
    else:
        df.loc[i, 'round'] = 15

In [1122]:
qb_df = df[df['pos']=='QB']
rb_df = df[df['pos']=='RB']
wr_df = df[df['pos']=='WR']
te_df = df[df['pos']=='TE']

In [1123]:
qb_df.drop(columns=['rk','pos','tgt','rec','rec_yards','y/r','rec_tds','standard_pts','vbd', 'team'], inplace=True)

In [1124]:
qb_df['rating'] = round(((((((qb_df['cmp']/qb_df['pass_att'])-.3)*5) + 
                  ((qb_df['pass_yds']/qb_df['pass_att']-3)*.25) +
                  ((qb_df['pass_tds']/qb_df['pass_att'])*20) +
                  (2.375-((qb_df['int']/qb_df['pass_att'])*25)))/6)*100),2)

In [1125]:
def add_target(group):
    group['target'] = group['ppr_pts'].shift(-1)
    return group

In [1126]:
qb_df = add_target(qb_df)


In [1134]:
qb_df = qb_df[(qb_df['player']!= 'Tom Brady') & (qb_df['player']!='Marcus Mariota') & (qb_df['player']!='Trace McSorley')]

In [1128]:
qb_df['comp%'] = round((qb_df['cmp'] / qb_df['pass_att']) * 100, 2)
qb_df['int%'] = round((qb_df['int'] / qb_df['pass_att']) * 100, 2)

In [1129]:
qb_df.drop(columns=['cmp','pass_att', 'int'], inplace=True)

In [1141]:
qb_cols = ['age','g','gs','pass_yds','pass_tds','rush_att','rush_yard','y/a','rush_tds',
           'fmb','fl','rush_rec_tds','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round','ppr_pts',
           'comp%', 'int%', 'rating']

In [1138]:
def xgb_modeling(df, cols):
    
    X_train = df[df['year']<2021]
    X_val = df[df['year']<2022]
    X_test = df[df['year']==2022]
    
    y_train = X_train['target']
    X_train.drop(columns=['target', 'year'], inplace = True)
    
    y_val = X_val['target']
    X_val.drop(columns=['target', 'year'], inplace = True)

    X_test.drop(columns=['target','year'], inplace = True)
    
    ss = StandardScaler()
    
    X_train[cols] = ss.fit_transform(X_train[cols])
    X_val[cols] = ss.transform(X_val[cols])
    X_test[cols] = ss.transform(X_test[cols])
    
    cols.append('success')
    
    xgb = xgboost.XGBRegressor(eval_metric='rmse',n_estimators=100, max_depth=4, 
                               subsample=.65, colsample_bytree=.8, seed=42,
                               eta=.15, gamma=100)
    
    xgb.fit(X_train[cols], y_train)
    xgb_preds = xgb.predict(X_train[cols])
    
    preds = pd.DataFrame({'actual':y_train,
                          'baseline':y_train.mean(),
                          'xgb_preds':xgb_preds})
    
    xgb_val_preds = xgb.predict(X_val[cols])
    
    val_preds = pd.DataFrame({'actual':y_val,
                              'baseline':y_train.mean(),
                              'xgb_val_preds':xgb_val_preds})
    
    
    pos_2022 = pd.DataFrame({'player':X_test['player'],
                             'preds':xgb.predict(X_test[cols])})
    
    return preds, val_preds, pos_2022

In [1118]:
qb_inverse_cols = ['pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [1032]:
for col in qb_inverse_cols:
    qb_df[col] = qb_df[col] * -1

In [1142]:
qb_preds, qb_val_preds, qb_2022 = xgb_modeling(qb_df, qb_cols)

In [1143]:
qb_2022.sort_values('preds',ascending=False).head(20)

,player,preds
2562,Jalen Hurts,366.929962
2561,Josh Allen,355.304474
2560,Patrick Mahomes,353.888000
2563,Joe Burrow,334.587158
2564,Geno Smith,290.891937
2566,Trevor Lawrence,285.931488
2568,Daniel Jones,281.837067
2565,Justin Fields,281.824432
2567,Kirk Cousins,278.568146
2569,Jared Goff,276.015564


In [1038]:
qb_df

,player,age,g,gs,pass_yds,pass_tds,rush_att,rush_yard,y/a,rush_tds,fmb,fl,rush_rec_tds,ppr_pts,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round,rating,target,comp%,int%
0,Aaron Rodgers,33,16,16,4428.0,40.0,67.0,369.0,5.51,4.0,8.0,4.0,4,380.0,1,2016,32.3,38.7,2,1.0,4.0,0 104.187158 1 117.127029 2 ...,347.5,65.74,1.15
1,Matt Ryan,31,16,16,4944.0,38.0,35.0,117.0,3.34,0.0,4.0,2.0,0,347.5,2,2016,183.0,146.5,20,1.0,13.0,0 104.187158 1 117.127029 2 ...,332.3,69.85,1.31
2,Drew Brees,37,16,16,5208.0,37.0,23.0,20.0,0.87,2.0,5.0,4.0,2,332.3,3,2016,57.2,61.1,5,1.0,6.0,0 104.187158 1 117.127029 2 ...,307.7,69.99,2.23
3,Andrew Luck,27,15,15,4240.0,31.0,64.0,341.0,5.33,2.0,6.0,5.0,2,307.7,4,2016,54.8,58.3,4,1.0,5.0,0 104.187158 1 117.127029 2 ...,300.3,63.49,2.39
4,Kirk Cousins,28,16,16,4917.0,25.0,34.0,96.0,2.82,4.0,9.0,3.0,4,300.3,5,2016,110.4,109.6,13,1.0,10.0,0 104.187158 1 117.127029 2 ...,286.9,67.00,1.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,Case Keenum,34,2,0,8.0,0.0,5.0,0.0,0.00,0.0,0.0,0.0,0,0.3,75,2022,457.0,514.0,43,0.0,15.0,0 104.187158 1 117.127029 2 ...,-0.4,28.57,0.00
3136,Nate Sudfeld,29,2,0,0.0,0.0,5.0,-4.0,-0.80,0.0,0.0,0.0,0,-0.4,76,2022,536.0,566.0,63,0.0,15.0,0 104.187158 1 117.127029 2 ...,-0.5,NaN,NaN
3137,Chad Henne,37,3,0,0.0,0.0,5.0,-5.0,-1.00,0.0,1.0,0.0,0,-0.5,77,2022,475.0,530.0,47,0.0,15.0,0 104.187158 1 117.127029 2 ...,-1.0,0.00,0.00
3138,C.J. Beathard,29,4,0,35.0,0.0,4.0,-4.0,-1.00,0.0,0.0,0.0,0,-1.0,78,2022,508.0,553.0,58,0.0,15.0,0 104.187158 1 117.127029 2 ...,-2.9,63.64,9.09


In [945]:
rmse(qb_preds,'baseline')

103.58817928105007

In [946]:
rmse(qb_preds, 'xgb_preds')

4.1313980366833745

In [947]:
rmse(qb_val_preds, 'xgb_val_preds')

41.44751640011753

In [948]:
rb_df.head()

,player,rk,team,pos,age,g,gs,cmp,pass_att,pass_yds,pass_tds,int,rush_att,rush_yard,y/a,rush_tds,tgt,rec,rec_yards,y/r,rec_tds,fmb,fl,rush_rec_tds,standard_pts,ppr_pts,vbd,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round
40,David Johnson,1,ARI,RB,25,16,16,0.0,0.0,0.0,0.0,0.0,293.0,1239.0,4.23,16.0,120.0,80.0,879.0,10.99,4.0,5.0,3.0,20,328.0,407.8,191.0,1,2016,5.4,3.4,1,1.0,1.0
41,Ezekiel Elliott,2,DAL,RB,21,15,15,0.0,0.0,0.0,0.0,0.0,322.0,1631.0,5.07,15.0,39.0,32.0,363.0,11.34,1.0,5.0,1.0,16,293.0,325.4,156.0,2,2016,6.3,8.3,3,1.0,1.0
42,Le'Veon Bell,5,PIT,RB,24,12,12,0.0,1.0,0.0,0.0,0.0,261.0,1268.0,4.86,7.0,94.0,75.0,616.0,8.21,2.0,4.0,1.0,9,242.0,317.4,105.0,3,2016,10.1,12.3,5,1.0,2.0
43,LeSean McCoy,4,BUF,RB,28,15,15,0.0,0.0,0.0,0.0,0.0,234.0,1267.0,5.41,13.0,57.0,50.0,356.0,7.12,1.0,3.0,0.0,14,248.0,298.3,111.0,4,2016,19.7,22.0,7,1.0,2.0
44,DeMarco Murray,6,TEN,RB,28,16,16,1.0,2.0,10.0,1.0,0.0,293.0,1287.0,4.39,9.0,67.0,53.0,377.0,7.11,3.0,3.0,1.0,12,241.0,293.8,104.0,5,2016,36.5,40.3,11,1.0,4.0


In [949]:
rb_df.drop(columns=['rk','team','pos','cmp','pass_att','pass_yds','pass_tds','int','standard_pts','vbd'], inplace=True)

In [950]:
rb_df = add_target(rb_df)

In [951]:
rb_cols = ['age','g','gs','rush_att','rush_yard','y/a','rush_tds','tgt','rec','rec_yards','y/r','rec_tds','fmb',
           'fl','rush_rec_tds','ppr_pts','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [952]:
rb_df

,player,age,g,gs,rush_att,rush_yard,y/a,rush_tds,tgt,rec,rec_yards,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round,target
40,David Johnson,25,16,16,293.0,1239.0,4.23,16.0,120.0,80.0,879.0,10.99,4.0,5.0,3.0,20,407.8,1,2016,5.4,3.4,1,1.0,1.0,325.4
41,Ezekiel Elliott,21,15,15,322.0,1631.0,5.07,15.0,39.0,32.0,363.0,11.34,1.0,5.0,1.0,16,325.4,2,2016,6.3,8.3,3,1.0,1.0,317.4
42,Le'Veon Bell,24,12,12,261.0,1268.0,4.86,7.0,94.0,75.0,616.0,8.21,2.0,4.0,1.0,9,317.4,3,2016,10.1,12.3,5,1.0,2.0,298.3
43,LeSean McCoy,28,15,15,234.0,1267.0,5.41,13.0,57.0,50.0,356.0,7.12,1.0,3.0,0.0,14,298.3,4,2016,19.7,22.0,7,1.0,2.0,293.8
44,DeMarco Murray,28,16,16,293.0,1287.0,4.39,9.0,67.0,53.0,377.0,7.11,3.0,3.0,1.0,12,293.8,5,2016,36.5,40.3,11,1.0,4.0,284.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2719,Mike Davis,29,8,0,8.0,18.0,2.25,0.0,2.0,2.0,4.0,2.00,0.0,1.0,0.0,0,4.2,119,2022,230.0,241.0,65,0.0,15.0,3.8
2720,Michael Burton,30,17,0,5.0,7.0,1.40,0.0,3.0,2.0,11.0,5.50,0.0,0.0,0.0,0,3.8,120,2022,417.0,424.0,108,0.0,15.0,2.0
2721,Keith Smith,30,17,6,1.0,2.0,2.00,0.0,2.0,1.0,8.0,8.00,0.0,0.0,0.0,0,2.0,121,2022,385.0,380.0,104,0.0,15.0,1.2
2722,Jerome Ford,23,13,0,8.0,12.0,1.50,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0,1.2,122,2022,349.0,368.0,100,0.0,15.0,0.3


In [953]:
rb_preds, rb_val_preds, rb_2022 = xgb_modeling(rb_df, rb_cols)

In [954]:
rmse(rb_preds,'baseline')

86.25716062741155

In [955]:
rmse(rb_preds,'xgb_preds')

3.6137978825602404

In [956]:
rmse(rb_val_preds,'xgb_val_preds')

18.884419875914713

In [957]:
rb_2022.sort_values('preds',ascending=False).head(20)

,player,preds
2601,Austin Ekeler,338.042999
2602,Christian McCaffrey,323.434479
2603,Josh Jacobs,304.960693
2604,Derrick Henry,279.762024
2606,Nick Chubb,267.002991
2605,Saquon Barkley,263.568817
2607,Rhamondre Stevenson,248.157623
2609,Aaron Jones,240.591354
2608,Tony Pollard,238.673798
2610,Joe Mixon,238.192337


In [958]:
wr_df.drop(columns=['rk','team','pos','cmp','pass_tds','pass_att','pass_yds','int','rush_att','rush_yard','y/a','rush_tds',
                    'standard_pts','vbd'],inplace=True)

In [959]:
wr_df = add_target(wr_df)

In [960]:
wr_df.head()

,player,age,g,gs,tgt,rec,rec_yards,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round,target
157,Antonio Brown,28,15,15,154.0,106.0,1284.0,12.11,12.0,0.0,0.0,12,307.3,1,2016,1.3,1.2,1,1.0,1.0,304.7
158,Jordy Nelson,31,16,16,152.0,97.0,1257.0,12.96,14.0,1.0,1.0,14,304.7,2,2016,19.3,17.3,10,1.0,2.0,304.1
159,Mike Evans,23,16,16,173.0,96.0,1321.0,13.76,12.0,0.0,0.0,12,304.1,3,2016,15.2,15.7,8,1.0,2.0,296.6
160,Odell Beckham Jr.,24,16,16,169.0,101.0,1367.0,13.53,10.0,3.0,1.0,10,296.6,4,2016,4.0,3.4,3,1.0,1.0,273.8
161,T.Y. Hilton,27,16,16,155.0,91.0,1448.0,15.91,6.0,0.0,0.0,6,273.8,5,2016,31.5,29.4,16,1.0,3.0,259.9


In [961]:
wr_cols = ['age','g','gs','tgt','rec','rec_yards','y/r','rec_tds','fmb','fl','rush_rec_tds','ppr_pts','pos_rank',
           'avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [962]:
wr_inverse_cols = ['fmb','fl','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [963]:
for col in wr_inverse_cols:
    wr_df[col] = wr_df[col] * -1

In [964]:
wr_preds, wr_val_preds, wr_2022 = xgb_modeling(wr_df, wr_cols)

In [965]:
rmse(wr_preds, 'baseline')

81.93125487276465

In [966]:
rmse(wr_preds,'xgb_preds')

2.367263839046847

In [967]:
rmse(wr_val_preds,'xgb_val_preds')

7.6258063581031195

In [968]:
wr_2022.sort_values('preds',ascending=False).head(20)

,player,preds
2727,Stefon Diggs,316.191528
2725,Tyreek Hill,310.495331
2888,KaVontae Turpin,308.045166
2726,Davante Adams,306.367401
2729,A.J. Brown,296.314026
2728,CeeDee Lamb,296.247131
2724,Justin Jefferson,286.046356
2730,Amon-Ra St. Brown,262.542816
2731,Jaylen Waddle,255.964661
2732,DeVonta Smith,244.230179


In [969]:
te_df.drop(columns=['rk','team','pos','cmp','pass_att','pass_yds','pass_tds','int','rush_att','rush_yard','y/a',
                    'rush_tds','standard_pts','vbd'], inplace=True)

In [970]:
te_df = add_target(te_df)

In [971]:
te_df.head()

,player,age,g,gs,tgt,rec,rec_yards,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round,target
307,Travis Kelce,27,16,15,117.0,85.0,1125.0,13.24,4.0,0.0,0.0,4,223.0,1,2016,61.4,64.3,6,1.0,6.0,209.0
308,Kyle Rudolph,27,16,16,132.0,83.0,840.0,10.12,7.0,0.0,0.0,7,209.0,2,2016,339.0,322.0,26,1.0,15.0,207.3
309,Greg Olsen,31,16,16,129.0,80.0,1073.0,13.41,3.0,0.0,0.0,3,207.3,3,2016,51.3,48.4,4,1.0,5.0,189.3
310,Jimmy Graham,30,16,15,95.0,65.0,923.0,14.20,6.0,2.0,2.0,6,189.3,4,2016,124.0,144.3,16,1.0,13.0,188.1
311,Delanie Walker,32,15,9,102.0,65.0,800.0,12.31,7.0,1.0,0.0,7,188.1,5,2016,68.9,64.0,5,1.0,6.0,183.6


In [972]:
te_cols = ['age','g','gs','tgt','rec','rec_yards','y/r','rec_tds','fmb','fl','rush_rec_tds','ppr_pts','pos_rank',
           'avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [973]:
te_inverse_cols = ['fmb','fl','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [974]:
for col in te_inverse_cols:
    te_df[col] = te_df[col] * -1

In [975]:
te_preds, te_val_preds, te_2022 = xgb_modeling(te_df, te_cols)

In [976]:
rmse(te_preds, 'baseline')

59.64335782390677

In [977]:
rmse(te_preds, 'xgb_preds')

2.854569461037304

In [978]:
rmse(te_val_preds,'xgb_val_preds')

9.502783798333281

In [979]:
te_2022.sort_values('preds',ascending=False).head(30)

,player,preds
2889,Travis Kelce,277.276672
2890,T.J. Hockenson,209.072006
2992,Feleipe Franks,207.062027
2891,George Kittle,192.006775
2892,Mark Andrews,182.686081
2893,Evan Engram,172.209747
2895,Pat Freiermuth,146.591980
2894,Tyler Higbee,145.883072
2896,Cole Kmet,144.962585
2898,Dalton Schultz,137.261887
